In [ ]:
import pandas as pd
import os
from datetime import datetime
from google.colab import drive
 
# 1. Mount Google Drive
drive.mount('/content/drive')
 
# Get the current week number
week_number = datetime.now().isocalendar()[1]
 
# 2. Define File Paths
# Assuming the corrected file naming conventions found during troubleshooting
poc_file_path = f'/content/drive/MyDrive/POC/POC_week_{week_number}.xlsx'
key_reversal_file_path = f'/content/drive/MyDrive/Key reversal/key_reversal_week_{week_number}.xlsx'
# Update the path for Divergenze RSI based on previous findings or assuming a corrected name
# If the directory is still not found, this will raise a FileNotFoundError
divergenze_rsi_file_path = f'/content/drive/MyDrive/DivergenzeRSI/Divergenze Rsi settimana {week_number}.xlsx' # Assuming this is the correct name based on previous attempts
 
# 3. Read Excel Files
df_poc = None
df_key_reversal = None
df_divergenze_rsi = None
 
try:
    df_poc = pd.read_excel(poc_file_path)
    print("Successfully read POC file.")
except FileNotFoundError:
    print(f"Error: POC file not found at {poc_file_path}")
 
try:
    df_key_reversal = pd.read_excel(key_reversal_file_path)
    print("Successfully read Key Reversal file.")
except FileNotFoundError:
    print(f"Error: Key Reversal file not found at {key_reversal_file_path}")
 
try:
    df_divergenze_rsi = pd.read_excel(divergenze_rsi_file_path)
    print("Successfully read Divergenze RSI file.")
    # Add print statement to check the Divergenze RSI DataFrame
    if df_divergenze_rsi is not None:
        print("\nDivergenze RSI DataFrame head:")
        # Removed display(df_divergenze_rsi.head()) to avoid JavaScript dependency
        print(df_divergenze_rsi.head().to_string())
except FileNotFoundError:
    print(f"Error: Divergenze RSI file not found at {divergenze_rsi_file_path}")
except Exception as e:
    print(f"An error occurred while reading Divergenze RSI file: {e}")
 
 
# 4. Process "Key reversal" and "Divergenze RSI" (if DataFrames were loaded)
if df_key_reversal is not None and not df_key_reversal.empty:
    # Ensure 'Date' column exists before processing
    if 'Date' in df_key_reversal.columns:
        df_key_reversal['Date'] = pd.to_datetime(df_key_reversal['Date'])
        df_key_reversal = df_key_reversal.sort_values(by=['Ticker', 'Date'], ascending=[True, False])
        df_key_reversal = df_key_reversal.drop_duplicates(subset='Ticker', keep='first')
        print("Processed Key Reversal DataFrame.")
    else:
        print("Warning: 'Date' column not found in Key Reversal file. Skipping date-based processing.")
else:
    print("Key Reversal DataFrame not loaded or is empty. Skipping processing.")
 
 
if df_divergenze_rsi is not None and not df_divergenze_rsi.empty:
    # Ensure 'Date2' column exists and process it as date
    if 'Date2' in df_divergenze_rsi.columns:
        df_divergenze_rsi['Date2'] = pd.to_datetime(df_divergenze_rsi['Date2'])
        # Use 'Date2' for sorting and dropping duplicates
        df_divergenze_rsi = df_divergenze_rsi.sort_values(by=['Ticker', 'Date2'], ascending=[True, False])
        df_divergenze_rsi = df_divergenze_rsi.drop_duplicates(subset='Ticker', keep='first')
        print("Processed Divergenze RSI DataFrame.")
    else:
        print("Warning: 'Date2' column not found in Divergenze RSI file. Skipping date-based processing.")
else:
    print("Divergenze RSI DataFrame not loaded or is empty. Skipping processing.")
 
 
# 5. Merge DataFrames
df_merged = df_poc # Start with the POC DataFrame
 
if df_key_reversal is not None and not df_key_reversal.empty and 'Ticker' in df_merged.columns and 'Ticker' in df_merged.columns:
    df_key_reversal_subset = df_key_reversal[['Ticker', 'Date', 'Signal']].rename(columns={'Date': 'Date_Key_Reversal', 'Signal': 'Signal_Key_Reversal'})
    df_merged = pd.merge(df_merged, df_key_reversal_subset, on='Ticker', how='left')
    print("Merged with Key Reversal DataFrame.")
elif df_key_reversal is None or df_key_reversal.empty:
     print("Skipping merge with Key Reversal: DataFrame not loaded or is empty.")
elif 'Ticker' not in df_merged.columns or 'Ticker' not in df_key_reversal.columns:
     print("Skipping merge with Key Reversal: 'Ticker' column not found in one of the DataFrames.")
 
 
if df_divergenze_rsi is not None and not df_divergenze_rsi.empty and 'Ticker' in df_merged.columns and 'Ticker' in df_merged.columns:
     df_divergenze_rsi_subset = df_divergenze_rsi[['Ticker', 'Date2', 'Mode']].rename(columns={'Date2': 'Date_Divergenze_RSI', 'Mode': 'Signal_Divergenze_RSI'})
     df_merged = pd.merge(df_merged, df_divergenze_rsi_subset, on='Ticker', how='left')
     print("Merged with Divergenze RSI DataFrame.")
elif df_divergenze_rsi is None or df_divergenze_rsi.empty:
     print("Skipping merge with Divergenze RSI: DataFrame not loaded or is empty.")
elif 'Ticker' not in df_merged.columns or 'Ticker' not in df_divergenze_rsi.columns:
     print("Skipping merge with Divergenze RSI: 'Ticker' column not found in one of the DataFrames.")
 
 
# 6. Output Merged Data
if df_merged is not None:
    # Filter for rows where at least one signal is present
    df_filtered = df_merged.dropna(subset=['Signal_Key_Reversal', 'Signal_Divergenze_RSI'], how='all')
    # Select only the desired columns for output
    output_columns = ['Ticker', 'Indice', 'Distanza POC %', 'Max Drawdown %', 'Avg Drawdown %', 'Current Drawdown %', 'Date_Key_Reversal', 'Signal_Key_Reversal', 'Date_Divergenze_RSI', 'Signal_Divergenze_RSI']
    df_filtered_output = df_filtered[output_columns]
    print("\nFiltered Merged DataFrame (Tickers with at least one signal):")
    # Ensure using print for text output
    print(df_filtered_output.to_string())
else:
    print("\nNo data to display after merging.")